### 시계열 데이터로서의 해석

In [ ]:
import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# 공유자동차 운행이력 데이터 불러오기
raw_data = pd.read_csv('/data/공유자동차예약이력.csv')

raw_data['Date'] = pd.to_datetime(raw_data['start_dt'])
data = pd.DataFrame(raw_data.groupby('Date').count()['ndevice_id'])
data = data.rename(columns={'ndevice_id':'rse_cnt'}).reset_index()

plt.figure(figsize=(22,8))
plt.plot(data.Date, data.rse_cnt)
plt.title('Reservation count')
plt.xlabel('Date')
plt.ylabel('reserv')

plt.show()

In [ ]:
# 데이터 시간 정리 -> 2021-01-04 부터 2022-06-27 사용 (이후 5일 예측)
ts_train = data[150:659]
ts_test = data[659:664]
ts_test2 = data[659:665]

timeSeries = ts_train.loc[:,['Date','rse_cnt']]
timeSeries.index = timeSeries.Date
ts = timeSeries.drop('Date', axis=1)


result = seasonal_decompose(ts['rse_cnt'], model='additive', period=12)
fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20,15)

In [ ]:
# ACF, PACF 그래프
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts[1:], lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts[1:], lags=20, ax=ax2)

In [ ]:
# ADF 검정
result = adfuller(ts)
print('ADF StatisticL %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values: ')
for key, value in result[4].items():
  print('\t%s: %.3f' %(key,value))

# => p-value < 0.05이므로 만족

In [ ]:
# KPSS 검정
def kpss_test(df):
  statistic, p_value, n_lags, critical_values = kpss(df.values)
  print(f'KPSS Statistic: {statistic}')
  print(f'p-value: {p_value}')
  print(f'num lags: {n_lags}')
  print('Critial Values:')
  for key, value in critical_values.items():
    print(f'{key} : {value}')

kpss_test(ts)

# => p-value < 0.05 이므로 정상시계열 아님

In [ ]:
# 차분그래프
ts_diff = ts - ts.shift()
plt.figure(figsize=(22,8))
plt.plot(ts_diff)
plt.title('Differencing method')
plt.xlabel('Date')
plt.ylabel('Differencing Reservation count')

plt.show()

result = adfuller(ts_diff[1:])
print('ADF StatisticL %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values: ')
for key, value in result[4].items():
  print('\t%s: %.3f' %(key,value))
# 1차 차분 데이터는 만족

kpss_test(ts_diff[1:])

# => p-value > 0.05 이므로 정상시계열

In [ ]:
# ACF, PACF 그래프
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_diff[1:], lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_diff[1:], lags=20, ax=ax2)

In [ ]:
# ARIMA(4, 1, 4)
model = ARIMA(ts, order=(4,1,4))
model_fit = model.fit(disp=0)

start_index = datetime(2022, 4, 1)
end_index = datetime(2022, 6, 27)
forecast = model_fit.predict(start=start_index, end=end_index, typ='levels')

plt.figure(figsize=(22,8))
plt.plot(ts_train.Date, ts_train.rse_cnt, label='original')
plt.plot(forecast, label='predicted')
plt.title('time series forecast')
plt.xlabel('Date')
plt.ylabel('Reservation')
plt.legend()

plt.show()

In [ ]:
# 잔차 분석
resi = np.array(ts_train[ts_train.Date>=start_index].rse_cnt) - np.array(forecast)

plt.figure(figsize=(22,8))
plt.plot(ts_train.Date[ts_train.Date>=start_index], resi)
plt.xlabel('Date')
plt.ylabel('residual')
plt.legend()

plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resi, lags=20, ax=ax1)

result = adfuller(resi
                  )
print('ADF StatisticL %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values: ')

for key, value in result[4].items():
  print('\t%s: %.3f' %(key,value))
# model로 적절

In [ ]:
print(model_fit.summary())

In [ ]:
# ARIMA (4, 0, 4)
model = ARIMA(ts, order=(4,0,4))
model_fit = model.fit(disp=0)

start_index = datetime(2022, 4, 1)
end_index = datetime(2022, 6, 27)
forecast = model_fit.predict(start=start_index, end=end_index, typ='levels')

plt.figure(figsize=(22,8))
plt.plot(ts_train.Date, ts_train.rse_cnt, label='original')
plt.plot(forecast, label='predicted')
plt.title('time series forecast')
plt.xlabel('Date')
plt.ylabel('Reservation')
plt.legend()

plt.show()

In [ ]:
# 잔차 분석
resi = np.array(ts_train[ts_train.Date>=start_index].rse_cnt) - np.array(forecast)

plt.figure(figsize=(22,8))
plt.plot(ts_train.Date[ts_train.Date>=start_index], resi)
plt.xlabel('Date')
plt.ylabel('residual')
plt.legend()

plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resi, lags=20, ax=ax1)

result = adfuller(resi)

print('ADF StatisticL %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values: ')

for key, value in result[4].items():
  print('\t%s: %.3f' %(key,value))

# p-value가 0.05보다 작음 => 모델로 적합

In [ ]:
#학습데이터셋으로부터 5일 뒤 예측
forecast_data = model_fit.forecast(steps=5)
pred_y = forecast_data[0].tolist()      # 5일의 예측데이터
test_y = ts_test.rse_cnt.values         # 실제 5일 데이터

pred_y_lower = []
pred_y_upper = []

for lower_upper in forecast_data[2]:
  lower = lower_upper[0]
  upper = lower_upper[1]
  pred_y_lower.append(lower)
  pred_y_upper.append(upper)

plt.plot(pred_y, color='gold')          # 모델 예측 그래프
plt.plot(pred_y_lower, color='red')     # 모델 예측 최저 그래프
plt.plot(pred_y_upper, color='blue')    # 모델 예측 최고 그래프
plt.plot(test_y, color='green')         # 실제

In [ ]:
#학습데이터셋으로부터 6일 뒤 예측
forecast_data = model_fit.forecast(steps=6)
pred_y = forecast_data[0].tolist()      # 6일의 예측데이터
test_y = ts_test2.rse_cnt.values        # 실제 6일 데이터

pred_y_lower = []
pred_y_upper = []

for lower_upper in forecast_data[2]:
  lower = lower_upper[0]
  upper = lower_upper[1]
  pred_y_lower.append(lower)
  pred_y_upper.append(upper)

plt.plot(pred_y, color='gold')          # 모델 예측 그래프
plt.plot(pred_y_lower, color='red')     # 모델 예측 최저 그래프
plt.plot(pred_y_upper, color='blue')    # 모델 예측 최고 그래프
plt.plot(test_y, color='green')         # 실제